# Credit score classification using logistic regression 

We are using a credit-related dataset found in Kaggle (source: https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data). The dataset consists of 27 input features and one target label, that is the credit score.

Goal : To predict the credit score of a new, unseen data using logistic regression model.

In [3]:
import numpy as np
np.random.seed(42)
import pandas as pd
import tqdm
import time

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.impute import KNNImputer
from numba import njit

np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Load and clean the data

In [4]:
def load_data():
    filename = "../train_data/cs_train.csv"
    df = pd.read_csv(filename)

    ## Separate target from features
    X = df.drop(columns=['Credit_Score'])
    y = df['Credit_Score']

    return X, y

In [5]:
def slice_split(X, y, num_samples, compare_to_r_ref):
    X = X.to_numpy()[:num_samples]
    y = y.to_numpy()[:num_samples]
    
    ## Split into training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1332)
    
    print(f"Train X shape is: {X_train.shape}")
    print(f"Train Y shape is: {y_train.shape}")
    print(f"Test X shape is: {X_test.shape}")
    print(f"Test Y shape is: {y_test.shape}")
    return X_train, X_test, y_train, y_test

In [6]:
X, y = load_data()
X.head()

/tmp/ipykernel_31302/566418612.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177


We'll then perform data cleaning to filter out irrelevant features for our prediction. A little bit of feature engineering after might be necessary.

First of all, we need to look more into our 'Monthly_Balance' column, which holds more than one data type, hence detected as DTypeWarning. 

In [7]:
print(X['Monthly_Balance'].unique())

['312.49408867943663' '284.62916249607184' '331.2098628537912' ...
 516.8090832742814 319.1649785257098 393.6736955618808]


In [8]:
X['Monthly_Balance'] = pd.to_numeric(X['Monthly_Balance'], errors='coerce')
X.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.494089
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.629162
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.209863
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.451310
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.489231


In [9]:
X.shape

(100000, 27)

'ID', 'Customer_ID', 'Name', 'SSN', 'Occupation', and 'Type_of_Loan' are, for now, **irrelevant** to the prediction.

In [10]:
## Drop irrelevant features
X = X.drop(columns=['ID', 'Customer_ID', 'Name', 'SSN', 'Occupation', 'Type_of_Loan'])
X.shape

(100000, 21)

Furthermore, there are features that hold string values, which obviously cannot be processed by the logistic regression model. We need to convert them into one-hot encoding vectors.

In [11]:
## One-hot encoding
X = pd.get_dummies(X, columns=['Month', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour'])
y = pd.get_dummies(y)
print(f"New shape of X: {X.shape}")
print(f"New shape of y: {y.shape}")

New shape of X: (100000, 39)
New shape of y: (100000, 3)


In [12]:
## Map 'Credit_History_Age' into decimals and NaN values
import re

# Replace 'NA' with NaN
X['Credit_History_Age'].replace('NA', pd.NA, inplace=True)

# Function to extract years and months
def extract_years_months(value):
    if pd.isna(value):
        return value
    match = re.match(r'(\d+)\s+Years?\s+and\s+(\d+)\s+Months?', value)
    if match:
        years, months = map(int, match.groups())
        return years + months / 12
    return None

# Apply the function to the column
X['Credit_History_Age'] = X['Credit_History_Age'].apply(extract_years_months)

/tmp/ipykernel_31302/2337723389.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['Credit_History_Age'].replace('NA', pd.NA, inplace=True)


In [13]:
X['Credit_History_Age'].head()

0    22.083333
1          NaN
2    22.250000
3    22.333333
4    22.416667
Name: Credit_History_Age, dtype: float64

Now, we need to address NaN values as a part of data cleaning process.

In [14]:
## Make sure all values are numerical
X_num = X.apply(pd.to_numeric, errors='coerce')

In [15]:
X_num.isna().sum()

Age                                                    4939
Annual_Income                                          6980
Monthly_Inhand_Salary                                 15002
Num_Bank_Accounts                                         0
Num_Credit_Card                                           0
Interest_Rate                                             0
Num_of_Loan                                            4785
Delay_from_due_date                                       0
Num_of_Delayed_Payment                                 9746
Changed_Credit_Limit                                   2091
Num_Credit_Inquiries                                   1965
Outstanding_Debt                                       1009
Credit_Utilization_Ratio                                  0
Credit_History_Age                                     9030
Total_EMI_per_month                                       0
Amount_invested_monthly                                8784
Monthly_Balance                         

In [16]:
## Fill the missing values with mean in each column
X_num = X_num.fillna(X_num.mean())

# Initialize the KNN imputer
# imputer = KNNImputer(missing_values=np.nan, n_neighbors=5)

# # Fit and transform the data
# start_time = time.time()
# X_imputed = imputer.fit_transform(X_num)
# end_time = time.time()

# print(f"KNNImputer fitting time: {end_time - start_time} seconds")

# # Convert back to DataFrame
# X_num = pd.DataFrame(X_imputed, columns=X_num.columns)
# del X_imputed

In [17]:
X_num.isna().sum()

Age                                                   0
Annual_Income                                         0
Monthly_Inhand_Salary                                 0
Num_Bank_Accounts                                     0
Num_Credit_Card                                       0
Interest_Rate                                         0
Num_of_Loan                                           0
Delay_from_due_date                                   0
Num_of_Delayed_Payment                                0
Changed_Credit_Limit                                  0
Num_Credit_Inquiries                                  0
Outstanding_Debt                                      0
Credit_Utilization_Ratio                              0
Credit_History_Age                                    0
Total_EMI_per_month                                   0
Amount_invested_monthly                               0
Monthly_Balance                                       0
Month_April                                     

## Feature engineering

Feature engineering lets us create new features by combining existing ones. These new features help the model to see the relationship between features and can improve the model's performance.

In [18]:
## Interaction features
X_num['Debt_to_Income_Ratio'] = X_num['Outstanding_Debt'] / X_num['Annual_Income']
X_num['Loan_to_Income_Ratio'] = X_num['Num_of_Loan'] / X_num['Annual_Income']
X_num['Investment_Ratio'] = X_num['Amount_invested_monthly'] / X_num['Monthly_Inhand_Salary']
X_num['Credit_Card_Utilization'] = X_num['Credit_Utilization_Ratio'] * X_num['Num_Credit_Card']

## Aggregated features
X_num['Total_Debt'] = X_num['Outstanding_Debt'] + X_num['Total_EMI_per_month']

## One-hot encoding for seasons
## --> Winter
X_num['Winter'] = X_num['Month_January'] + X_num['Month_February']
## --> Spring
X_num['Spring'] = X_num['Month_March'] + X_num['Month_April'] + X_num['Month_May']
## --> Summer
X_num['Summer'] = X_num['Month_June'] + X_num['Month_July'] + X_num['Month_August']

We also need to perform normalization to some features so that all features are in a virtually comparable range of values (feature scaling). We will use z-score normalization that subtracts the mean from each feature and divide the resulting value by its standard deviation.

First, we will reduce the impact of **outliers** (i.e., skewed, and possibly incorrect,
values in columns that might distort z-score normalization results) by applying natural log transformation.

In [19]:
columns_log = [
    'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
    'Num_Credit_Card',
    'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment',
    'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt',
    'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month',
    'Amount_invested_monthly', 'Monthly_Balance',
    'Debt_to_Income_Ratio', 'Loan_to_Income_Ratio', 'Investment_Ratio', 
    'Credit_Card_Utilization', 'Total_Debt'
]

## Prevent FloatingPointError
X_num[columns_log] = X_num[columns_log].map(lambda x: x if x >= 0 else 0.01)

## Apply log transformation to necessary columns
X_num[columns_log] = np.log1p(X_num[columns_log])

We can then proceed for normalization.

In [20]:
## We want to normalize all columns except one-hot encoding vectors
columns_norm = [
    'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card',
    'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment',
    'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt',
    'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month',
    'Amount_invested_monthly', 'Monthly_Balance',
    'Debt_to_Income_Ratio', 'Loan_to_Income_Ratio', 'Investment_Ratio', 
    'Credit_Card_Utilization', 'Total_Debt'
]

## Initialize the scaler
scaler = StandardScaler()

## Fit and transform the selected columns
X_num[columns_norm] = scaler.fit_transform(X_num[columns_norm])

In [21]:
X_num['Debt_to_Income_Ratio'].head()

0   -0.1441
1   -0.1441
2   -0.1441
3   -0.1441
4   -0.1441
Name: Debt_to_Income_Ratio, dtype: float64

In [22]:
X_num['Num_Credit_Card'].head()

0   -0.409512
1   -0.409512
2   -0.409512
3   -0.409512
4   -0.409512
Name: Num_Credit_Card, dtype: float64

## Risk minimization

We need to come to an understanding that logistic regression model cannot be trained on the target label of three possible binary values.

In [23]:
y.head()

,Good,Poor,Standard
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False


Let's assume that the global finance company we're working with is not looking for more market expansion and is trying to minimize loan risk. Therefore, we want to label parties with **Good** credit scores as being eligible for loans, while the rest are not.

In [24]:
## Create a new binary label
y['Target'] = y['Good']

## Drop the original one-hot encoded columns
y = y.drop(columns=['Good', 'Standard', 'Poor'])

In [25]:
y.shape

(100000, 1)

## Using feature selection

Use feature selection to reduce dimension.

In [24]:
def apply_selection(X, y, k):
    selector = SelectKBest(f_classif, k=k)
    y_new = y.values.ravel()
    X_new = pd.DataFrame(selector.fit_transform(X, y_new), columns=X.columns[selector.get_support()])

    print(f"Selected features shape: {X_new.shape}")
    return X_new

In [25]:
X_fs = apply_selection(X_num, y, 32)

Selected features shape: (100000, 32)


## Preparing data for training

In [85]:
## NUM_SAMPLES is the combination of all sets (NUM_TRAINING + NUM_TEST)
NUM_SAMPLES = 2048
COMPARE_TO_R_REF = False
lr = 0.1
mu = 0.1

## Slice data into NumPy arrays and split into training and test sets
X_train, X_test, y_train, y_test = slice_split(
    X_num, y,
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)

## n = num_of_features
n = X_train.shape[1]

# Same shape as Marcelo's reference code
betas = np.zeros((n, ))
# betas = np.random.randn(n)

Train X shape is: (1024, 47)
Train Y shape is: (1024, 1)
Test X shape is: (1024, 47)
Test Y shape is: (1024, 1)


In [86]:
## See ratio between negative and positive samples
num_neg = np.sum(y_train == 0)  
num_pos = np.sum(y_train == 1)  
print(f"Number of negative samples: {num_neg}")
print(f"Number of positive samples: {num_pos}")
print(f"Ratio neg/pos: {num_neg/num_pos}")

Number of negative samples: 818
Number of positive samples: 206
Ratio neg/pos: 3.970873786407767


In [76]:
n

47

## Applying Principal Component Analysis

Principal Component Analysis (PCA) allows for a reduced dataset, meaning the new dataset will have a small number of new features (called the principal components), which explains all original features. 

How well these new features explain the original ones is parameterized by explained variance ratio.

In [ ]:
from sklearn.decomposition import PCA

def apply_pca(X_train, X_test, n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Number of components after PCA: {X_train_pca.shape[1]}")
    
    return X_train_pca, X_test_pca

In [ ]:
X_train, X_test = apply_pca(X_train, X_test, n_components=32)

## n = num_of_features
n = X_train.shape[1]
print(f"n = {n}")

# Same shape as Marcelo's reference code
betas = np.zeros((n, ))

In [ ]:
print(f"Train X shape is: {X_train.shape}")
print(f"Train Y shape is: {y_train.shape}")
print(f"Test X shape is: {X_test.shape}")
print(f"Test Y shape is: {y_test.shape}")

## Nesterov model training

In [77]:
@njit
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

@njit
def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas   # A vector of linear_predictions/logits z = train_x @ weights
    if dbg:
        print(f"Length of Logits: {len(preds)}")
    return np.expand_dims(sigmoid(preds), -1)   # Shape: (m, 1)

@njit
def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)   # A vector of logistic predictions y_hat = sigmoid(z)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient   # Shape: (10, 1) == Rows correspond to num_features
    ## This function is used to update the values of betas: w_new = w_old + lr * gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))   # h: hypothesis, basically preds/y_hat
    t1 = np.matmul(-y.T, np.log(np.clip(h, 0.000000000000001, np.max(h))))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    ## np.clip() function prevents computing the log of 0, by taking the minimum of 1e-15.
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]   # Shape: (1,) == scalar value

def nesterov(betas, epochs, patience, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    check_early = 0
    best_loss = float('inf')
    best_epochs_list = []

    nesterov_loss = [0 for _ in range(epochs)]
    for i in tqdm.trange(epochs):
    # for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)

        ## Assign updated weights into phi_prime
        phi_prime = theta - lr * np.squeeze(gradient)   # np.squeeze() removes single dimensions --> shape (10,)
        
        ## Nesterov acceleration process
        if i == 0:
            theta = phi_prime
        else:
            ## If current updated weight (phi_prime) < previous weight (phi), 
            ## The updated weight theta will be even smaller.
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime   # phi is then the weight of the previous epoch/update if Nesterov is used

        # ## Ditching Nesterov for now
        # theta = phi_prime   
        loss = cost(train_x, train_y, theta)

        ## Early stopping
        if patience > 0:
            if loss < best_loss:
                best_loss = loss
                best_epoch = i
                best_theta = theta
                best_epochs_list.append(best_epoch)
                check_early = 0
            else:
                check_early += 1
                if check_early >= patience:
                    print(f"Early stopping at {patience} epochs after {best_epoch} with best loss {best_loss}")
                    return nesterov_loss[:best_epoch + 1], best_theta, phi, best_epochs_list
        else:
            best_epoch = epochs - 1
            best_theta = theta

        ## Update list
        nesterov_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return nesterov_loss[:best_epoch + 1], best_theta, phi, best_epochs_list

def sgd(betas, epochs, patience, lr, train_x, train_y):
    import copy
    theta = copy.deepcopy(betas)

    np.random.seed(42)

    check_early = 0
    best_loss = float('inf')
    best_epochs_list = []

    sgd_loss = [0 for _ in range(epochs)]
    for i in tqdm.trange(epochs):
        ## Shuffle training data for each epoch
        indices = np.random.permutation(train_x.shape[0])

        for j in indices:
            x_j = train_x[j:j+1]
            y_j = train_y[j:j+1]

            gradient = calculate_gradient(x_j, y_j, theta, fwd, dbg=False)

            ## Assign updated weights into phi_prime
            phi_prime = theta - lr * np.squeeze(gradient)   # np.squeeze() removes single dimensions --> shape (10,)

            theta = phi_prime   # SGD does not use Nesterov acceleration
        
        loss = cost(train_x, train_y, theta)

        ## Early stopping
        if patience > 0:
            if loss < best_loss:
                best_loss = loss
                best_epoch = i
                best_theta = theta
                best_epochs_list.append(best_epoch)
                check_early = 0
            else:
                check_early += 1
                if check_early >= patience:
                    print(f"Early stopping at {patience} epochs after {best_epoch} with best loss {best_loss}")
                    return sgd_loss[:best_epoch + 1], best_theta, best_epochs_list
        else:
            best_epoch = epochs - 1
            best_theta = theta

        ## Update list
        sgd_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return sgd_loss[:best_epoch + 1], best_theta, best_epochs_list

def minibgd(betas, epochs, patience, lr, train_x, train_y, batch_size=32):
    import copy
    theta = copy.deepcopy(betas)

    check_early = 0
    best_loss = float('inf')
    best_epochs_list = []

    mbgd_loss = [0 for _ in range(epochs)]
    for i in tqdm.trange(epochs):

        for j in range(0, train_x.shape[0], batch_size):
            batch_x = train_x[j:j+batch_size]
            batch_y = train_y[j:j+batch_size]

            gradient = calculate_gradient(batch_x, batch_y, theta, fwd, dbg=False)

            ## Assign updated weights into phi_prime
            phi_prime = theta - lr * np.squeeze(gradient)   # np.squeeze() removes single dimensions --> shape (10,)

            theta = phi_prime   # SGD does not use Nesterov acceleration
        
        loss = cost(train_x, train_y, theta)

        ## Early stopping
        if patience > 0:
            if loss < best_loss:
                best_loss = loss
                best_epoch = i
                best_theta = theta
                best_epochs_list.append(best_epoch)
                check_early = 0
            else:
                check_early += 1
                if check_early >= patience:
                    print(f"Early stopping at {patience} epochs after {best_epoch} with best loss {best_loss}")
                    return mbgd_loss[:best_epoch + 1], best_theta, best_epochs_list
        else:
            best_epoch = epochs - 1
            best_theta = theta

        ## Update list
        mbgd_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return mbgd_loss[:best_epoch + 1], best_theta, best_epochs_list


In [78]:
# Ensure inputs are of the correct type
betas = np.array(betas, dtype=np.float64)
X_train = np.array(X_train, dtype=np.float64)
y_train = np.array(y_train, dtype=np.float64)
X_test = np.array(X_test, dtype=np.float64)
y_test = np.array(y_test, dtype=np.float64)
lr = float(lr)
mu = float(mu)

In [79]:
# losses, theta, phi, best_epochs_list = nesterov(betas, 200, -1, lr, mu, X_train, y_train)
# losses, theta, best_epochs_list = sgd(betas, 200, -1, 0.01, X_train, y_train)
losses, theta, best_epochs_list = minibgd(betas, 200, -1, lr, X_train, y_train, batch_size=32)

100%|██████████| 200/200 [00:01<00:00, 151.27it/s]


In [80]:
for j in range(len(losses)):
    print(f"{j}: {losses[j]}", end='\n')
# Ideal Gradient (800 epochs) #
## seed(42), lr = 0.001, mu = 0.3 --> loss = 0.5433165928958901
## seed(42), lr = 0.001, mu = 0.5 --> loss = 0.5139704562064134

# Oscillating Gradient (800 epochs) #
## seed(42), lr = 0.003, mu = 0.5 --> loss = 0.49509424569059735
## seed(42), lr = 0.003, mu = 0.6 --> loss = 0.4102733269497619

# Early stopping (patience = 50) #
## seed(42), lr = 0.003, mu = 0.5 --> 964: 0.410228534339208
## seed(42), lr = 0.003, mu = 0.6 --> 422: 0.46334269494781627

# Ideal Gradient (800 epochs) with Feature Engineering #
## seed(42), lr = 0.001, mu = 0.3 --> loss = 0.523951850559082
## seed(42), lr = 0.001, mu = 0.5 --> loss = 0.4922252143522289

# Early stopping (patience = 50) with Feature Engineering #
## seed(42), lr = 0.003, mu = 0.5 --> 739: 0.41164361016071044
## seed(42), lr = 0.003, mu = 0.6 --> 314: 0.4688941580652026

0: 0.42125106019865904
1: 0.3693794905971943
2: 0.3478817434046736
3: 0.3360000116419437
4: 0.32829000621174265
5: 0.32278365708616535
6: 0.318604228099489
7: 0.3152993904858073
8: 0.31260913963261205
9: 0.31037131716377847
10: 0.3084782733762296
11: 0.3068550096406488
12: 0.30544726565806873
13: 0.30421459506926485
14: 0.30312611758549113
15: 0.30215779204371856
16: 0.3012905973746678
17: 0.3005092790937982
18: 0.2998014612715736
19: 0.29915700239436105
20: 0.29856751859097247
21: 0.2980260245504902
22: 0.2975266589880438
23: 0.29706447199646524
24: 0.29663525844593974
25: 0.29623542614501086
26: 0.29586189057857637
27: 0.29551199019538477
28: 0.29518341774424944
29: 0.294874164256499
30: 0.2945824730739405
31: 0.2943068019146685
32: 0.2940457914129708
33: 0.2937982389054721
34: 0.29356307649227553
35: 0.2933393525996698
36: 0.2931262164246643
37: 0.2929229047619365
38: 0.29272873080859263
39: 0.29254307461734436
40: 0.2923653749286931
41: 0.29219512216082344
42: 0.29203185237469026
4

In [37]:
for j in range(len(best_epochs_list)):
    print(f"{j}: {best_epochs_list[j]}", end='\n')

In [38]:
theta

array([ 0.1437495 ,  0.03777745,  0.11725458, -0.17135686, -0.17610091,
        0.07257256, -0.42476234, -0.32293601,  0.02841478, -0.05726835,
       -0.09527048, -0.1234824 ,  0.04065134,  0.15703102,  0.27538635,
        0.0701872 ,  0.13025743, -0.19517463, -0.17656722, -0.25464855,
       -0.23849051, -0.09356381, -0.15213365, -0.15677825, -0.16665692,
       -0.3194744 ,  0.22000199, -0.93197905, -0.40256208, -0.25322962,
       -0.21853602, -0.9622479 , -0.13740634, -0.20394414, -0.25258636,
       -0.09026297, -0.20241603, -0.16243453, -0.38496316,  0.08435517,
       -0.15761871, -0.07988609, -0.26570654,  0.1459805 , -0.49313906,
       -0.51860981, -0.42226468])

## Model performance on training set

We can test the performance of the model through its confusion matrix, F1 score, and accuracy score.

First, we predict on the training set.

In [39]:
## On training data
pred = fwd(X_train, theta, dbg=False)

## Decision (Threshold = 0.5)
y_train_hat = (pred >= 0.5).astype(int)

In [40]:
y_train_hat.shape

(1024, 1)

In [41]:
y_train.shape

(1024, 1)

In [42]:
print(confusion_matrix(y_train, y_train_hat))

[[771  47]
 [ 98 108]]


In [43]:
print(f1_score(y_train, y_train_hat))

0.5983379501385041


In [44]:
print(precision_score(y_train, y_train_hat))

0.6967741935483871


In [45]:
print(recall_score(y_train, y_train_hat))

0.5242718446601942


In [46]:
print(roc_auc_score(y_train, pred))

0.8983549742445462


In [47]:
print(accuracy_score(y_train, y_train_hat))

0.8583984375


## Model performance on test set

Now, we predict on the test set.

In [81]:
## On test data

pred = fwd(X_test, theta, dbg=False)

## Decision (Threshold = 0.5)
y_test_hat = (pred >= 0.5).astype(int)

In [49]:
y_test_hat.shape

(1024, 1)

In [50]:
y_test.shape

(1024, 1)

In [51]:
print(confusion_matrix(y_test, y_test_hat))

[[753  60]
 [107 104]]


In [52]:
print(f1_score(y_test, y_test_hat))

0.5546666666666666


In [53]:
print(roc_auc_score(y_test, pred))

0.8737226234821591


In [54]:
print(accuracy_score(y_test, y_test_hat))

0.8369140625


In [55]:
print(precision_score(y_test, y_test_hat))

0.6341463414634146


In [56]:
print(recall_score(y_test, y_test_hat))

0.4928909952606635


Sweep F1 score across different threshold values.

In [82]:
thresholds = np.linspace(0, 1, 101)
f1s = []

for t in thresholds:
    y_test_hat_sweep = (pred >= t).astype(int)
    f1s.append(f1_score(y_test, y_test_hat_sweep))

best_t = thresholds[np.argmax(f1s)]
best_f1 = max(f1s)

print(f"Best threshold: {best_t:.2f}, Best F1: {best_f1:.3f}")

Best threshold: 0.27, Best F1: 0.634


## Write to a CSV file

In [52]:
# Convert the NumPy array to a DataFrame
X_train_csv = pd.DataFrame(X_train)
y_train_csv = pd.DataFrame(y_train)
X_test_csv = pd.DataFrame(X_test)
y_test_csv = pd.DataFrame(y_test)

# Save the DataFrame to a CSV file
X_train_csv.to_csv("../cscore_data/X_train_1024fs.csv", index=False)
y_train_csv.to_csv("../cscore_data/y_train_1024fs.csv", index=False)
X_test_csv.to_csv("../cscore_data/X_test_1024fs.csv", index=False)
y_test_csv.to_csv("../cscore_data/y_test_1024fs.csv", index=False)

In [53]:
del X

In [54]:
X_train_csv[X_train_csv.columns[:]].head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.460089,0.319067,0.636022,0.771718,-0.172056,0.847819,0.408311,0.007451,0.447042,1.104410,...,1.0,0.0,1.0,0.0,-0.324485,-0.410564,0.173646,0.423971,0.0,1.0
1,-0.366288,0.305226,0.626379,-0.003572,0.202622,-0.866501,1.034754,0.143579,0.004386,0.744293,...,1.0,1.0,0.0,0.0,-0.479415,-0.409154,-0.012949,0.025207,0.0,1.0
2,0.360107,-0.174785,-0.108077,-1.408773,0.028710,-0.711485,0.408311,-0.145735,-1.047162,0.288454,...,0.0,1.0,0.0,0.0,-0.395609,-0.895605,0.022229,-0.236370,0.0,0.0
3,0.081902,-0.089745,0.040337,0.771718,0.202622,0.292650,0.110895,0.855789,0.673314,-1.003964,...,1.0,0.0,0.0,1.0,-0.066335,1.138855,0.480004,0.454371,0.0,1.0
4,-0.049991,0.992729,1.489341,-0.236773,0.202622,-1.563541,0.408311,-0.641981,-0.543955,-2.285441,...,0.0,1.0,0.0,0.0,-0.555817,-1.146745,0.414829,0.262347,0.0,1.0
